In [1]:
%pip install gurobipy
%pip install pandas xlrd
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 78.9 MB/s eta 0:00:00


In [ ]:
import re
import pandas as pd
import numpy as np


# Example R output string
r_output = """
## 1) root 80 42207120.0 4474.154
## 2) System 1: Heating Source=DX Coils 56 9451800.0 4139.805
## 4) Heater Fuel=Gas 12 761177.5 3726.917 *
## 5) Heater Fuel=Electricity 44 6086973.0 4252.411
## 10) Roof: Ext Finish/Color=Aluminum Paint,Uncolored 32 1204252.0 4078.056
## 20) AGW: Exterior Insulation>=5 23 237098.3 3991.113 *
## 21) AGW: Exterior Insulation< 5 9 348985.9 4300.244 *
## 11) Roof: Ext Finish/Color=Dark,Light 12 1315822.0 4717.358 *
## 3) System 1: Heating Source=Electric Resistance 24 11888070.0 5254.300
## 6) Heater Fuel=Gas 12 2210965.0 4666.217 *
## 7) Heater Fuel=Electricity 12 1376897.0 5842.383 *
"""

# Regular expression patterns for extracting information
pattern = r"([0-9]+)+\)\s?(.*?(?:(?:[<=])|(?:[> ])|(?:[=])).*)\s(\d+)\s([\d.]+)\s([\d.]+)\s(\*)?"

# Extract information from the R output string
matches = re.findall(pattern, r_output)

# Create a dictionary to store the extracted information
data = {
    'Node-id': [],
    'Left_child': [],
    'Right_child': [],
    'Split': [],
    'Count': [],
    'Risk': [],
    'Prediction': [],
    'nodetype': []

}

# Parse the extracted information and populate the dictionary
for match in matches:
    node, split, count, risk, prediction, nodetype = match
    # nodetype='terminalnode' if  nodetype=='*' else "splitnode"
    nodetype = 'terminalnode' if nodetype == '*' else ("root" if split.strip() == 'root' else 'splitnode')
    left_child= int(node)*2       if  nodetype!='terminalnode'   else -2
    right_child=int(node)*2+1     if  nodetype!='terminalnode'   else -2
    data['Node-id'].append(int(node))
    data['Left_child'].append(int(left_child))
    data['Right_child'].append(int(right_child))
    data['Split'].append(split)
    data['Count'].append(int(count))
    data['Risk'].append(float(risk))
    data['Prediction'].append(float(prediction))
    data['nodetype'].append(nodetype)


# Create a pandas DataFrame from the dictionary
df = pd.DataFrame(data)
# print(df)

In [ ]:
n_nodes =len(df['Node-id'].tolist())
node_id=df['Node-id'].tolist()
children_left=df['Left_child'].tolist()
children_right=df['Right_child'].tolist()
# feature=tree_dataframe['feature'].tolist()
node_type=df['nodetype'].tolist()
criteria = df['Split'].tolist()
value = df['Prediction'].tolist()
n_samples= df['Count'].tolist()
paths = []
path = []

def recurse(node, path, paths):
  node_index=df[df['Node-id'] == node].index.item()
  if node_type[node_index]!='terminalnode':

       CriteriaL=criteria[df[df['Node-id'] == children_left[node_index]].index.item()]
       CriteriaR=criteria[df[df['Node-id'] == children_right[node_index]].index.item()]

       p1, p2 = list(path), list(path)

       p1 +=[CriteriaL]

       recurse(children_left[node_index], p1, paths)
       p2 +=[CriteriaR]
       recurse(children_right[node_index], p2, paths)
  else:
      path += [(f"Prediction={value[node_index]}", f"Terminalnode={node}")]
      paths += [path]
      # print(path)

recurse(1, path, paths)
print(paths)


TerminalNode_ID=[node_id[index] for index, element in enumerate(node_type) if element == 'terminalnode']




[['System 1: Heating Source=DX Coils', 'Heater Fuel=Gas', ('Prediction=3726.917', 'Terminalnode=4')], ['System 1: Heating Source=DX Coils', 'Heater Fuel=Electricity', 'Roof: Ext Finish/Color=Aluminum Paint,Uncolored', 'AGW: Exterior Insulation>=5', ('Prediction=3991.113', 'Terminalnode=20')], ['System 1: Heating Source=DX Coils', 'Heater Fuel=Electricity', 'Roof: Ext Finish/Color=Aluminum Paint,Uncolored', 'AGW: Exterior Insulation< 5', ('Prediction=4300.244', 'Terminalnode=21')], ['System 1: Heating Source=DX Coils', 'Heater Fuel=Electricity', 'Roof: Ext Finish/Color=Dark,Light', ('Prediction=4717.358', 'Terminalnode=11')], ['System 1: Heating Source=Electric Resistance', 'Heater Fuel=Gas', ('Prediction=4666.217', 'Terminalnode=6')], ['System 1: Heating Source=Electric Resistance', 'Heater Fuel=Electricity', ('Prediction=5842.383', 'Terminalnode=7')]]


In [ ]:
from google.colab import files

uploaded = files.upload()
# Read the Excel file into a DataFrame
import io
Factors=pd.read_excel('Vishnu_variables.xlsx', header=0)
# df = pd.read_excel('Runs2023-2022.xlsx', header=0)
#df = df.drop('Unnamed: 0', axis=1)
# Access the column headings
headings_Factors = Factors.columns
# headings_df = df.columns

Factors.head()
# print(headings_df)

Saving Vishnu_variables.xlsx to Vishnu_variables (3).xlsx


,variable,description,type,values
0,1,Roof: Ext Finish/Color,Categorical,"""Light"", ""Dark"", ""Uncolored"", ""Aluminum Paint"""
1,2,Roof: Exterior Insulation,Numerical-discrete,"0, 4, 36, 35, 24, 20, 12, 9, 30, 28, 42, 18, 1..."
2,3,Roof: Add'l Insulation,Numerical-discrete,"0, 19, 49, 38, 11, 3, 30, 13, 60, 21, 7, 15, 26"
3,4,AGW: Ext Finish/Color,Categorical,"""Light"", ""Dark"", ""Uncolored"", ""Medium"""
4,5,AGW: Exterior Insulation,Numerical-discrete,"10.5, 21, 2, 1.3, 7, 14, 4, 12, 0, 18, 6, 9, 8"


In [ ]:

Var_lib = {}
Var_name_type={}
Var_index_type={}
Var_name_index={}

for row in Factors.itertuples(index=False):
    key = (str(row[0]), row[2])  # Tuple of first and third columns

    extracted_list=[]
    extracted_list = re.findall(r'"([^"]+)"', row[3])
    if not re.findall(r'"([^"]+)"', row[3]):
     extracted_list= extracted_list = row[3].split(", ")

    Var_lib[key[0]] = extracted_list
    Var_name_type[row[1]]=key[1]
    Var_index_type[key[0]]=key[1]
    Var_name_index[row[1]]=key[0]


print(Var_index_type)

result_cat = {}
result_num_disc = {}
result_num_cont_LB = {}
result_num_cont_UB = {}

for item in paths:


    split_index = len(item) - 1

    part1 = item[:split_index]
    part2 = item[-1]

    terminalnode=terminal_node = part2[1].split('=')[1]

    for entry in part1:

      variables = {}

      if re.findall(r"(.*?(?:(?:<\s)).*)", entry):

              variable, value = re.split('<\s', entry)
              variables[variable] = value.split(',')

              if Var_name_type.get(variable) == 'Numerical-continuous':
                 result_num_cont_U B[(terminalnode,variable)] = variables[variable]

              elif Var_name_type.get(variable) == 'Numerical-discrete':
                sortedvariables=sorted(Var_lib[Var_name_index[variable]])
                variables[variable]= sortedvariables[:int(variables[variable][0])]
                result_num_disc[(terminalnode,variable)] = variables[variable]

      if re.findall(r"(.*?(?<![>=])=.*)", entry):


              variable, value = re.split('=', entry)

              variables[variable] = value.split(',')
              result_cat[(terminalnode,variable)] = variables[variable]


      if re.findall(r"(.*?(?:(?:>=)).*)", entry):

              variable, value = re.split('>=', entry)
              variables[variable] = value.split(',')

              if Var_name_type.get(variable) == 'Numerical-continuous':
                 result_num_cont_LB[(terminalnode,variable)] = variables[variable]

              elif Var_name_type.get(variable) == 'Numerical-discrete':
                sortedvariables=sorted(Var_lib[Var_name_index[variable]])
                variables[variable]= sortedvariables[int(variables[variable][0]):]
                result_num_disc[(terminalnode,variable)] = variables[variable]



print(result_cat)
print(result_num_disc)
# print(result_num_cont_UB)

{'1': 'Categorical', '2': 'Numerical-discrete', '3': 'Numerical-discrete', '4': 'Categorical', '5': 'Numerical-discrete', '6': 'Numerical-discrete', '7': 'Numerical-discrete', '8': 'Numerical-discrete', '9': 'Numerical-discrete', '10': 'Categorical', '11': 'Numerical-discrete', '12': 'Categorical', '13': 'Categorical', '14': 'Categorical', '15': 'Categorical', '16': 'Categorical', '17': 'Categorical'}
{('4', 'System 1: Heating Source'): ['DX Coils'], ('4', 'Heater Fuel'): ['Gas'], ('20', 'System 1: Heating Source'): ['DX Coils'], ('20', 'Heater Fuel'): ['Electricity'], ('20', 'Roof: Ext Finish/Color'): ['Aluminum Paint', 'Uncolored'], ('21', 'System 1: Heating Source'): ['DX Coils'], ('21', 'Heater Fuel'): ['Electricity'], ('21', 'Roof: Ext Finish/Color'): ['Aluminum Paint', 'Uncolored'], ('11', 'System 1: Heating Source'): ['DX Coils'], ('11', 'Heater Fuel'): ['Electricity'], ('11', 'Roof: Ext Finish/Color'): ['Dark', 'Light'], ('6', 'System 1: Heating Source'): ['Electric Resistance'

In [ ]:
#Define Sets
# decision variables
Var_number=len(Var_index_type)

count_dict = {}

# Create an empty dictionary to store the counts
count_dict = {}

# Iterate through the elements in the dictionary
for valuetype in Var_index_type.values():
    # Get the second element of the tuple key
    second_element = valuetype

    # Check if the second element exists in the count dictionary
    if second_element in count_dict:
        # Increment the count for that second element
        count_dict[second_element] += 1
    else:
        # Add the second element to the count dictionary with a count of 1
        count_dict[second_element] = 1

count_dict


{'Categorical': 9, 'Numerical-discrete': 8}

In [ ]:


# numerical decision variables
J=[str(i) for i in range (1,Var_number)]
TerminalNode_count= node_type.count('terminalnode')

J_num=  [key for key, value in Var_index_type.items() if (value == 'Numerical-discrete' or value == 'Numerical-continuous')]
J_num_disc= [key for key, value in Var_index_type.items() if value == 'Numerical-discrete']
J_num_cont= [key for key  , value in Var_index_type.items() if  value == 'Numerical-continuous']

# Categorical decision variables
J_cat=[key for key, value in Var_index_type.items() if value == 'Categorical']


#response variables
R=['1']

#terminal nodes
R, T=gp.multidict({
    '1': [[node_id[index] for index, element in enumerate(node_type) if element == 'terminalnode']]
})




###Fill in the dictionaries

In [ ]:
#Input data
# Mean of the responses for those observations in the terminal nodes
# R tree, t terminal node

# Regression Intercept
Beta_Zero=[value[index] for index, element in enumerate(node_type) if element == 'terminalnode']
RT = [(key, str(value)) for key in T.keys() for value in T[key]]

RT, Beta_Zero=gp.multidict(dict(zip(RT, Beta_Zero)))
my_lib={key[0]: [value] for key, value in Var_lib.items() if key[0] in J_num_cont}

J_cat, cat_vls=gp.multidict({key: [value] for key  , value in Var_lib.items() if key in J_cat}) if J_cat else [], {}
J_num_disc, dic_vls=gp.multidict({key: [value] for key  , value in Var_lib.items() if key in J_num_disc}) if J_num_disc else [], {}
J_num_cont, con_vls = gp.multidict({key: [value] for key  , value in  Var_lib.items() if key in J_num_cont }) if J_num_cont else [], {}

print(dic_vls)


# LB={}
# for j in J_num_cont:
#  LB.update(dict.fromkeys([(j,r,t) for r in R for t in T[r] ], lw_b[j]) )

# UB={}
# for j in J_num:
#  UB.update(dict.fromkeys([(j,r,t) for r in R for t in T[r] ], up_b[j]) )

# For discrete variables, the values they can actually take
# the calculation for R-... Batt come from Marjan's thesis page 35-36
# J_num_disc, dic_vls=gp.multidict({
#     '1': [[2,4,6,8]],
#     '2': [[1,1.5,2,3]],
#     '3': [[1.25,2,3,4]],
#     '4': [[1,2,3,4]],
#     '5': [[1,1.5,2,3]],
#     '6': [[0.9677,2.2580,3.5484,4.1935]],
#     '7': [[0.1,0.15,0.2,0.25]],
#     '8': [[0.1,0.15,0.2,0.25]],
#     '9': [[0.1,0.15,0.2,0.25]],
#     '10': [[0.1,0.15,0.2,0.25]],
#     '11': [[2.2580,6.1290,6.7742,9.6774]],
#     '12': [[13,19,21,30]],
#     '13': [[1,1.5,2,3]],
#     '14': [[100,70.7,62.5,50]],
#     '15': [[3,4,5,6]],
#     '16': [[2,2.3,2.7,3]],
# })

DISC_Val={}
for j in J_num_disc:
 for v in range(1,len(dic_vls[j])+1):
  DISC_Val.update(dict.fromkeys([(j,str(v))], dic_vls[j][v-1]) )


Jdisc_V, DISC_ValS=gp.multidict(DISC_Val)

print(DISC_ValS)
# bounds of numerical variables
J_num_cont, lw_b, up_b=gp.multidict({
    '1':  [min(dic_vls['1']),max(dic_vls['1'])],
    '2':  [min(dic_vls['2']),max(dic_vls['2'])],
    '3':  [min(dic_vls['3']),max(dic_vls['3'])],
    '4':  [min(dic_vls['4']),max(dic_vls['4'])],
    '5':  [min(dic_vls['5']),max(dic_vls['5'])],
    '6':  [min(dic_vls['6']),max(dic_vls['6'])],
    '7':  [min(dic_vls['7']),max(dic_vls['7'])],
    '8':  [min(dic_vls['8']),max(dic_vls['8'])],
    '9':  [min(dic_vls['9']),max(dic_vls['9'])],
    '10': [min(dic_vls['10']),max(dic_vls['10'])],
    '11': [min(dic_vls['11']),max(dic_vls['11'])],
    '12': [min(dic_vls['12']),max(dic_vls['12'])],
    '13': [min(dic_vls['13']),max(dic_vls['13'])],
    '14': [min(dic_vls['14']),max(dic_vls['14'])],
    '15': [min(dic_vls['15']),max(dic_vls['15'])],
    '16': [min(dic_vls['16']),max(dic_vls['16'])],
    '17': [575,675],
    '18': [10,30],
    '19': [575,675],
    '20': [10,30],
    '21': [575,675],
    '22': [10,30],
    '23': [5,105],
    '24': [10,30],
    '25': [250,350],
    '26': [5,25],
    '27': [100,200],
    '28': [5,25],
    '29': [100,200],
    '30': [15,35],
    '31': [100,200],
    '32': [5,25],
})

#Initialize dictionaries for upper and lower bound of variables at each terminal load of each tree

LB={}
for j in J_num:
 LB.update(dict.fromkeys([(j,r,t) for r in R for t in T[r] ], lw_b[j]) )

UB={}
for j in J_num:
 UB.update(dict.fromkeys([(j,r,t) for r in R for t in T[r] ], up_b[j]) )

# Update Upper bound a lower bounds of variables accroding to Regression Tree (Refer to Dickson's article)

# limits of the variables in respose 1 tree

UB.update({('11','1','1'): 5})
UB.update({('11','1','2'): 5})
UB.update({('11','1','3'): 5})
UB.update({('11','1','4'): 5})
LB.update({('11','1','5'): 5})
LB.update({('11','1','6'): 5})
LB.update({('11','1','7'): 5})
LB.update({('11','1','8'): 5})
UB.update({('18','1','1'): 20})
LB.update({('18','1','2'): 20})
UB.update({('19','1','5'): 651.2})
UB.update({('19','1','6'): 651.2})
UB.update({('19','1','7'): 651.2})
LB.update({('19','1','8'): 651.2})
UB.update({('29','1','5'): 144.55})
LB.update({('29','1','6'): 144.55})
LB.update({('29','1','7'): 144.55})
UB.update({('30','1','6'): 23.985})
LB.update({('30','1','7'): 23.985})

# limits of the variables in respose 2 tree

UB.update({('23','2','1'): 104.6})
UB.update({('23','2','2'): 104.6})
UB.update({('23','2','3'): 104.6})
UB.update({('23','2','4'): 104.6})
UB.update({('23','2','5'): 104.6})
UB.update({('23','2','6'): 104.6})
LB.update({('23','2','7'): 104.6})
UB.update({('24','2','1'): 26.91})
UB.update({('24','2','2'): 26.91})
UB.update({('24','2','3'): 26.91})
UB.update({('24','2','4'): 26.91})
UB.update({('24','2','5'): 26.91})
LB.update({('24','2','6'): 26.91})
UB.update({('25','2','1'): 255.9})
LB.update({('25','2','2'): 255.9})
LB.update({('25','2','3'): 255.9})
LB.update({('25','2','4'): 255.9})
LB.update({('25','2','5'): 255.9})
UB.update({('19','2','2'): 654.3})
UB.update({('19','2','3'): 654.3})
UB.update({('19','2','4'): 654.3})
LB.update({('19','2','5'): 654.3})
UB.update({('23','2','2'): 29.6})
LB.update({('23','2','3'): 29.6})
LB.update({('23','2','4'): 29.6})
UB.update({('32','2','3'): 12.575})
LB.update({('32','2','4'): 12.575})

# limits of the variables in respose 3 tree

UB.update({('11','3','1'): 5})
UB.update({('11','3','2'): 5})
UB.update({('11','3','3'): 5})
UB.update({('11','3','4'): 5})
LB.update({('11','3','5'): 5})
LB.update({('11','3','6'): 5})
LB.update({('11','3','7'): 5})
LB.update({('11','3','8'): 5})
UB.update({('18','3','1'): 20})
LB.update({('18','3','2'): 20})
UB.update({('19','3','5'): 651.2})
UB.update({('19','3','6'): 651.2})
UB.update({('19','3','7'): 651.2})
LB.update({('19','3','8'): 651.2})
UB.update({('29','3','5'): 144.55})
LB.update({('29','3','6'): 144.55})
LB.update({('29','3','7'): 144.55})
UB.update({('30','3','6'): 23.985})
LB.update({('30','3','7'): 23.985})

jrt, l_jrt = gp.multidict(LB)
jrt, u_jrt=gp.multidict(UB)



R, D=gp.multidict({
    '1': [600],
    '2': [30],
    '3': [11000]
})

{}


TypeError: ignored